In [1]:
# Install the necessary libraries 
!pip install scikit-learn pandas tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

from google.colab import files
 
 
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
applicationtype_counts = application_df.APPLICATION_TYPE.value_counts()
applicationtype_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ...?
replace_application = list(applicationtype_counts[applicationtype_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [11]:
# Generate our categorical variable lists
applications_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
applications_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[applications_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(applications_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(applications_cat,1)
application_df.head()

<ipython-input-13-65e8842a67f6>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(applications_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=72)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
##ADDED Hidden layer with 10 nodes - second attempt at 75% accuracy
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_modelt2 = tf.keras.models.Sequential()

# First hidden layer
nn_modelt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_modelt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_modelt2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_modelt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_modelt2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [18]:
# Compile the model
nn_modelt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Create a callback that saves the model's weights every 5v epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [20]:
# Train the model
fit_model = nn_modelt2.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Streaming output truncated to the last 5000 lines.
444/804 [===============>..............] - ETA: 2s - loss: 0.5314 - accuracy: 0.7411
Epoch 85: saving model to checkpoints/weights.85.hdf5

Epoch 85: saving model to checkpoints/weights.85.hdf5
454/804 [===============>..............] - ETA: 2s - loss: 0.5316 - accuracy: 0.7411
Epoch 85: saving model to checkpoints/weights.85.hdf5
463/804 [================>.............] - ETA: 2s - loss: 0.5321 - accuracy: 0.7406
Epoch 85: saving model to checkpoints/weights.85.hdf5

Epoch 85: saving model to checkpoints/weights.85.hdf5
471/804 [================>.............] - ETA: 2s - loss: 0.5319 - accuracy: 0.7410
Epoch 85: saving model to checkpoints/weights.85.hdf5

Epoch 85: saving model to checkpoints/weights.85.hdf5
479/804 [================>.............] - ETA: 2s - loss: 0.5318 - accuracy: 0.7414
Epoch 85: saving model to checkpoints/weights.85.hdf5

Epoch 85: saving model to checkpoints/weights.85.hdf5
489/804 [=================>.......

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_modelt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5689 - accuracy: 0.7263 - 760ms/epoch - 3ms/step
Loss: 0.5688959956169128, Accuracy: 0.7262973785400391


In [22]:
# Export our model to HDF5 file
nn_modelt2.save("AlphabetSoupCharity.h5")

In [25]:
# Zip the checkpoints folder
!zip -r checkpoints.zip checkpoints/

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/weights.95.hdf5 (deflated 42%)
  adding: checkpoints/weights.85.hdf5 (deflated 42%)
  adding: checkpoints/weights.89.hdf5 (deflated 42%)
  adding: checkpoints/weights.49.hdf5 (deflated 42%)
  adding: checkpoints/weights.56.hdf5 (deflated 42%)
  adding: checkpoints/weights.38.hdf5 (deflated 42%)
  adding: checkpoints/weights.58.hdf5 (deflated 42%)
  adding: checkpoints/weights.17.hdf5 (deflated 42%)
  adding: checkpoints/weights.73.hdf5 (deflated 42%)
  adding: checkpoints/weights.83.hdf5 (deflated 42%)
  adding: checkpoints/weights.15.hdf5 (deflated 42%)
  adding: checkpoints/weights.65.hdf5 (deflated 42%)
  adding: checkpoints/weights.87.hdf5 (deflated 42%)
  adding: checkpoints/weights.46.hdf5 (deflated 42%)
  adding: checkpoints/weights.79.hdf5 (deflated 42%)
  adding: checkpoints/weights.14.hdf5 (deflated 42%)
  adding: checkpoints/weights.01.hdf5 (deflated 42%)
  adding: checkpoints/weights.70.hdf5 (deflated 42%)
  adding: c

In [27]:
# download the zipped checkpoints folder:
from google.colab import files
files.download('checkpoints.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
##Removed third hidden layer - adding more nodes to first and second hidden lyers and increasing epochs to 150  - third attempt at 75% accuracy
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 90
hidden_nodes_layer5 = 65

nn_modelt3 = tf.keras.models.Sequential()

# First hidden layer
nn_modelt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_modelt3.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn_modelt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_modelt3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 90)                3960      
                                                                 
 dense_5 (Dense)             (None, 65)                5915      
                                                                 
 dense_6 (Dense)             (None, 1)                 66        
                                                                 
Total params: 9,941
Trainable params: 9,941
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [30]:
# Compile the model
nn_modelt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Create a callback that saves the model's weights every 5v epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [32]:

# Train the model
fit_model = nn_modelt3.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Streaming output truncated to the last 5000 lines.
Epoch 135: saving model to checkpoints/weights.135.hdf5
448/804 [===============>..............] - ETA: 2s - loss: 0.5312 - accuracy: 0.7425
Epoch 135: saving model to checkpoints/weights.135.hdf5

Epoch 135: saving model to checkpoints/weights.135.hdf5
454/804 [===============>..............] - ETA: 2s - loss: 0.5315 - accuracy: 0.7420
Epoch 135: saving model to checkpoints/weights.135.hdf5

Epoch 135: saving model to checkpoints/weights.135.hdf5
464/804 [================>.............] - ETA: 2s - loss: 0.5316 - accuracy: 0.7417
Epoch 135: saving model to checkpoints/weights.135.hdf5
471/804 [================>.............] - ETA: 2s - loss: 0.5307 - accuracy: 0.7425
Epoch 135: saving model to checkpoints/weights.135.hdf5

Epoch 135: saving model to checkpoints/weights.135.hdf5
479/804 [================>.............] - ETA: 2s - loss: 0.5300 - accuracy: 0.7432
Epoch 135: saving model to checkpoints/weights.135.hdf5
487/804 [========

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_modelt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5745 - accuracy: 0.7256 - 1s/epoch - 4ms/step
Loss: 0.5745111703872681, Accuracy: 0.7255976796150208


In [34]:
# Export our model to HDF5 file
nn_modelt3.save("AlphabetSoupCharity.h5")

In [35]:
# Zip the checkpoints folder
!zip -r checkpoints.zip checkpoints/

updating: checkpoints/ (stored 0%)
updating: checkpoints/weights.95.hdf5 (deflated 31%)
updating: checkpoints/weights.85.hdf5 (deflated 31%)
updating: checkpoints/weights.89.hdf5 (deflated 31%)
updating: checkpoints/weights.49.hdf5 (deflated 31%)
updating: checkpoints/weights.56.hdf5 (deflated 31%)
updating: checkpoints/weights.38.hdf5 (deflated 31%)
updating: checkpoints/weights.58.hdf5 (deflated 31%)
updating: checkpoints/weights.17.hdf5 (deflated 31%)
updating: checkpoints/weights.73.hdf5 (deflated 31%)
updating: checkpoints/weights.83.hdf5 (deflated 31%)
updating: checkpoints/weights.15.hdf5 (deflated 31%)
updating: checkpoints/weights.65.hdf5 (deflated 31%)
updating: checkpoints/weights.87.hdf5 (deflated 31%)
updating: checkpoints/weights.46.hdf5 (deflated 31%)
updating: checkpoints/weights.79.hdf5 (deflated 31%)
updating: checkpoints/weights.14.hdf5 (deflated 31%)
updating: checkpoints/weights.01.hdf5 (deflated 31%)
updating: checkpoints/weights.70.hdf5 (deflated 31%)
updating: c

In [36]:
# download the zipped checkpoints folder:
from google.colab import files
files.download('checkpoints.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
##adding third layerand  more nodes to first and second hidden layers  - fourth attempt at 75% accuracy
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 15

nn_modelt4 = tf.keras.models.Sequential()

# First hidden layer
nn_modelt4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_modelt4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_modelt4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_modelt4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 200)               8800      
                                                                 
 dense_12 (Dense)            (None, 100)               20100     
                                                                 
 dense_13 (Dense)            (None, 1)                 101       
                                                                 
Total params: 29,001
Trainable params: 29,001
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [48]:
# Compile the model
nn_modelt4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
# Create a callback that saves the model's weights every 5v epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)
     

In [50]:
# Train the model
fit_model = nn_modelt4.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Streaming output truncated to the last 5000 lines.
Epoch 85: saving model to checkpoints/weights.85.hdf5

Epoch 85: saving model to checkpoints/weights.85.hdf5
449/804 [===============>..............] - ETA: 2s - loss: 0.5343 - accuracy: 0.7416
Epoch 85: saving model to checkpoints/weights.85.hdf5
457/804 [================>.............] - ETA: 2s - loss: 0.5346 - accuracy: 0.7412
Epoch 85: saving model to checkpoints/weights.85.hdf5
462/804 [================>.............] - ETA: 2s - loss: 0.5352 - accuracy: 0.7407
Epoch 85: saving model to checkpoints/weights.85.hdf5
467/804 [================>.............] - ETA: 2s - loss: 0.5347 - accuracy: 0.7410
Epoch 85: saving model to checkpoints/weights.85.hdf5

Epoch 85: saving model to checkpoints/weights.85.hdf5
474/804 [================>.............] - ETA: 2s - loss: 0.5344 - accuracy: 0.7414
Epoch 85: saving model to checkpoints/weights.85.hdf5
479/804 [================>.............] - ETA: 2s - loss: 0.5345 - accuracy: 0.7414
Epoch

In [51]:

# Evaluate the model using the test data
model_loss, model_accuracy = nn_modelt4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
     

268/268 - 1s - loss: 0.5768 - accuracy: 0.7271 - 564ms/epoch - 2ms/step
Loss: 0.5768027901649475, Accuracy: 0.7271137237548828


In [52]:
# Export our model to HDF5 file
nn_modelt4.save("AlphabetSoupCharity.h5")

In [53]:
# Zip the checkpoints folder
!zip -r checkpoints.zip checkpoints/

updating: checkpoints/ (stored 0%)
updating: checkpoints/weights.95.hdf5 (deflated 17%)
updating: checkpoints/weights.85.hdf5 (deflated 17%)
updating: checkpoints/weights.89.hdf5 (deflated 17%)
updating: checkpoints/weights.49.hdf5 (deflated 17%)
updating: checkpoints/weights.56.hdf5 (deflated 17%)
updating: checkpoints/weights.38.hdf5 (deflated 17%)
updating: checkpoints/weights.58.hdf5 (deflated 17%)
updating: checkpoints/weights.17.hdf5 (deflated 17%)
updating: checkpoints/weights.73.hdf5 (deflated 17%)
updating: checkpoints/weights.83.hdf5 (deflated 17%)
updating: checkpoints/weights.15.hdf5 (deflated 17%)
updating: checkpoints/weights.65.hdf5 (deflated 17%)
updating: checkpoints/weights.87.hdf5 (deflated 17%)
updating: checkpoints/weights.46.hdf5 (deflated 17%)
updating: checkpoints/weights.79.hdf5 (deflated 17%)
updating: checkpoints/weights.14.hdf5 (deflated 17%)
updating: checkpoints/weights.01.hdf5 (deflated 17%)
updating: checkpoints/weights.70.hdf5 (deflated 17%)
updating: c

In [54]:
# download the zipped checkpoints folder:
from google.colab import files
files.download('checkpoints.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>